# NER Model

In [ ]:
# Импортирование модулей сервиса
from ConversationManager import ConversationManager
from NLU_classifier      import NLU_Classifier

from transformers import pipeline
from transformers import pipeline, AutoModel, AutoTokenizer


class NerExtractor:
    """
    Класс для извлечения именованных сущностей (NER) из текста, использующий предобученные модели из библиотеки transformers.

    Атрибуты:
        token_pred_pipeline (pipeline): Пайплайн для классификации токенов, используемый для извлечения сущностей.

    Методы:
        concat_entities: Объединяет именованные сущности, принадлежащие к одному и тому же типу и расположенные рядом друг с другом.
        get_entities: Извлекает именованные сущности из предоставленного текста.
    """
    def __init__(self, model_checkpoint="./LaBSE_ner_nerel"):
        """
        Инициализирует экстрактор сущностей с использованием указанной модели.

        Параметры:
            model_checkpoint (str): Путь к предобученной модели NER.
        """
        print('Loading NER model...')
        # Загрузка токенизатора для предварительно обученной модели NER
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

        # Загрузка предварительно обученной модели NER с переносом на GPU для ускорения обработки
        self.model = AutoModel.from_pretrained(model_checkpoint).to(device='cuda')
        print('Retriver NER Loaded')

        # Инициализация пайплайна для классификации токенов
        self.token_pred_pipeline = pipeline("token-classification",
                                            model=self.model,
                                            tokenizer=self.tokenizer,
                                            aggregation_strategy="average",
                                            device='cuda')

    @staticmethod
    def concat_entities(ner_result):
        """
        Объединяет последовательные именованные сущности одного типа в одну сущность.

        Эта функция полезна для ситуаций, когда одна именованная сущность
        разделена на несколько частей моделью NER. Например, "Нью" и "Йорк"
        могут быть идентифицированы как отдельные сущности, но на самом деле
        это одна сущность "Нью-Йорк".

        Параметры:
            ner_result (list): Список именованных сущностей, возвращаемых пайплайном NER.

        Возвращает:
            list: Объединенный список именованных сущностей.
        """
        entities = []       # Инициализация списка для объединенных сущностей
        prev_entity = None  # Предыдущий тип сущности
        prev_end = 0        # Позиция окончания предыдущей сущности

        # Перебор результатов NER
        for i in range(len(ner_result)):
            # Проверка, является ли текущая сущность продолжением предыдущей
            if (ner_result[i]["entity_group"] == prev_entity) and (ner_result[i]["start"] == prev_end):
                # Обновление конечной позиции предыдущей сущности, объединяя ее с текущей
                entities[-1][2] = ner_result[i]["end"]
            else:
                # Добавление новой сущности в список
                entities.append([ner_result[i]["entity_group"],
                                 ner_result[i]["start"],
                                 ner_result[i]["end"]])
            # Обновление информации о предыдущей сущности
            prev_entity = ner_result[i]["entity_group"]
            prev_end = ner_result[i]["end"]

        return entities


    def get_entities(self, text: str):
        """
        Извлекает именованные сущности из текста.

        Эта функция использует предварительно обученный NER пайплайн для определения
        и извлечения именованных сущностей (например, имен, организаций, местоположений)
        из предоставленного текста.

        Параметры:
            text (str): Текст для извлечения сущностей.

        Возвращает:
            list: Список извлеченных сущностей.

        Ошибки:
            AssertionError: Если текст пустой.
        """
        # Проверка, что текст не пустой
        assert len(text) > 0, "Предоставленный текст пустой."

        # Использование NER пайплайна для извлечения сущностей из текста
        entities = self.token_pred_pipeline(text)

        # Объединение соседних именованных сущностей одного типа
        # Это полезно, когда одна сущность разбивается на несколько частей моделью NER
        concat_ent = self.concat_entities(entities)

        return concat_ent